In [1]:
from google.colab import files
uploaded = files.upload()

Saving box.zip to box.zip


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00


In [4]:
"""
GraphFNO for PCA-compressed Time Series Prediction on Graphs
Implements the exact Sp2GNO architecture from the paper:
"Spatio-Spectral Graph Neural Operator for Solving Computational Mechanics Problems"

FIXED: Variable Lipschitz embedding dimension issue
"""

import os
import glob
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import get_laplacian, to_dense_adj
import random
from tqdm import tqdm
import zipfile
import tempfile
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh

# ============================================================================
# HYPERPARAMETERS (ANTI-OVERFITTING CONFIGURATION)
# ============================================================================
ZIP_FILE_PATH = "/content/box.zip"
PCA_COMPONENTS = 20
WIDTH = 64  # Reduced from 128 to prevent overfitting
NUM_SP2GNO_BLOCKS = 3  # Reduced from 4 to simplify model
K_CHEBYSHEV = 6  # Chebyshev polynomial order
M_EIGENVECTORS = 16  # Reduced from 32 to prevent overfitting
K_NEIGHBORS = 20  # K for KNN graph construction
BATCH_SIZE = 2
LR = 5e-4  # Reduced learning rate for better generalization
EPOCHS = 3000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

# Regularization parameters
DROPOUT_RATE = 0.15  # Dropout for regularization
WEIGHT_DECAY = 1e-3  # L2 regularization (increased)
GRAD_CLIP = 0.5  # Gradient clipping (reduced for stability)
SMOOTHNESS_WEIGHT = 0.05  # Increased spatial smoothness regularization

def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# ============================================================================
# DATA LOADING UTILITIES
# ============================================================================

def extract_zip(zip_path, extract_to=None):
    if extract_to is None:
        extract_to = tempfile.mkdtemp()
    print(f"Extracting {zip_path} to {extract_to}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    extracted_items = os.listdir(extract_to)
    if len(extracted_items) == 1 and os.path.isdir(os.path.join(extract_to, extracted_items[0])):
        extract_to = os.path.join(extract_to, extracted_items[0])
    print(f"Extraction complete. Data root: {extract_to}")
    return extract_to

def load_case(case_dir):
    csv_p = os.path.join(case_dir, "node_feature.csv")
    txt_p = os.path.join(case_dir, "Label.txt")

    if not os.path.exists(csv_p):
        possible_csv = list(Path(case_dir).rglob("node_feature.csv"))
        if possible_csv:
            csv_p = str(possible_csv[0])
            case_dir = os.path.dirname(csv_p)
            txt_p = os.path.join(case_dir, "Label.txt")

    if not os.path.exists(csv_p):
        raise FileNotFoundError(f"Missing {csv_p}")
    if not os.path.exists(txt_p):
        raise FileNotFoundError(f"Missing {txt_p}")

    node_df = pd.read_csv(csv_p)
    valid_mask = node_df['id'] == 1
    df_valid = node_df[valid_mask].reset_index(drop=True)

    with open(txt_p, "r") as f:
        lines = [line.strip() for line in f if line.strip() != ""]

    labels = [np.fromstring(line, sep=' ') for line in lines]
    labels = np.vstack(labels)

    if df_valid.shape[0] != labels.shape[0]:
        raise ValueError(f"Mismatch: valid node rows {df_valid.shape[0]} vs label rows {labels.shape[0]}")

    feats = df_valid[['width','height','x','y','angle']].values.astype(float)
    coords = df_valid[['x','y']].values.astype(float)

    return node_df, feats, coords, labels, df_valid

def build_edge_index(n_nodes):
    if n_nodes == 0:
        return torch.empty((2,0), dtype=torch.long)
    rows = []
    cols = []
    for i in range(n_nodes):
        prev_neighbor = (i - 1) % n_nodes
        rows.append(i)
        cols.append(prev_neighbor)
        next_neighbor = (i + 1) % n_nodes
        rows.append(i)
        cols.append(next_neighbor)
    edge_index = torch.tensor([rows, cols], dtype=torch.long)
    return edge_index

def find_case_directories(data_root):
    case_dirs = []
    for root, dirs, files in os.walk(data_root):
        if 'node_feature.csv' in files and 'Label.txt' in files:
            case_dirs.append(root)

    if not case_dirs:
        potential_cases = [d for d in os.listdir(data_root)
                          if os.path.isdir(os.path.join(data_root, d))]
        for case in potential_cases:
            case_path = os.path.join(data_root, case)
            if (os.path.exists(os.path.join(case_path, 'node_feature.csv')) and
                os.path.exists(os.path.join(case_path, 'Label.txt'))):
                case_dirs.append(case_path)

    case_dirs = [os.path.relpath(case, data_root) for case in case_dirs]
    return sorted(case_dirs)

class PaperPCA:
    """PCA implementation matching the paper's approach"""
    def __init__(self, n_components):
        self.n_components = n_components
        self.mean_ = None
        self.components_ = None
        self.explained_variance_ = None

    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        X_centered = X - self.mean_
        cov_matrix = np.cov(X_centered, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        idx = np.argsort(eigenvalues)[::-1]
        self.components_ = eigenvectors[:, idx[:self.n_components]]
        self.explained_variance_ = eigenvalues[idx[:self.n_components]]

    def transform(self, X):
        X_centered = X - self.mean_
        return X_centered @ self.components_

    def inverse_transform(self, X_transformed):
        return X_transformed @ self.components_.T + self.mean_

# ============================================================================
# LIPSCHITZ EMBEDDINGS (Section 3.2.2 of paper)
# ============================================================================

def compute_lipschitz_embeddings(edge_index, num_nodes):
    """
    Compute Lipschitz positional embeddings as described in the paper.
    Uses shortest path distances from anchor nodes.
    Number of anchors n = O(log(N)^2) as per Bourgain's theorem.
    """
    if num_nodes <= 1:
        return torch.zeros((num_nodes, 1), dtype=torch.float32)

    # Number of anchors based on Bourgain's theorem
    n_anchors = max(1, int(np.log(max(num_nodes, 2)) ** 2))
    n_anchors = min(n_anchors, num_nodes)

    # Select anchor nodes randomly (use fixed seed for reproducibility)
    np.random.seed(42)
    anchor_indices = np.random.choice(num_nodes, size=n_anchors, replace=False)

    # Build adjacency list for shortest path computation
    adj_list = [[] for _ in range(num_nodes)]
    edge_index_np = edge_index.numpy()
    for i in range(edge_index_np.shape[1]):
        src, dst = edge_index_np[0, i], edge_index_np[1, i]
        if src != dst:  # Avoid self-loops
            adj_list[src].append(dst)

    # Compute shortest path distances from each anchor using BFS
    embeddings = np.zeros((num_nodes, n_anchors))

    for anchor_idx, anchor in enumerate(anchor_indices):
        # BFS from anchor
        distances = np.full(num_nodes, np.inf)
        distances[anchor] = 0
        queue = [anchor]
        visited = set([anchor])

        while queue:
            node = queue.pop(0)
            for neighbor in adj_list[node]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    distances[neighbor] = distances[node] + 1
                    queue.append(neighbor)

        # Handle unreachable nodes (set to max distance)
        max_dist = num_nodes
        distances[distances == np.inf] = max_dist
        embeddings[:, anchor_idx] = distances

    # Normalize embeddings to prevent numerical issues
    mean = embeddings.mean(axis=0, keepdims=True)
    std = embeddings.std(axis=0, keepdims=True) + 1e-8
    embeddings = (embeddings - mean) / std

    return torch.tensor(embeddings, dtype=torch.float32)

# ============================================================================
# GRAPH LAPLACIAN AND EIGENVECTORS (Section 3.2.1 of paper)
# ============================================================================

def compute_normalized_laplacian_eigenvectors(edge_index, num_nodes, m):
    """
    Compute first m eigenvectors of normalized graph Laplacian.
    Uses LOBPCG algorithm as mentioned in the paper for efficiency O(mE).
    Returns Q_m ∈ R^{N×m} containing first m eigenvectors.
    """
    if num_nodes <= 2:
        # For very small graphs, return identity
        m_actual = min(m, num_nodes)
        eigenvectors = torch.eye(num_nodes, m_actual, dtype=torch.float32)
        eigenvalues = torch.zeros(m_actual, dtype=torch.float32)
        return eigenvectors, eigenvalues

    # Build adjacency matrix
    edge_index_np = edge_index.numpy()
    row = edge_index_np[0]
    col = edge_index_np[1]
    data = np.ones(len(row))

    # Create sparse adjacency matrix
    adj = sp.coo_matrix((data, (row, col)), shape=(num_nodes, num_nodes))
    adj = adj.tocsr()

    # Make adjacency symmetric (undirected graph)
    adj = adj + adj.T
    adj = (adj > 0).astype(float)

    # Compute degree matrix
    degree = np.array(adj.sum(axis=1)).flatten()
    degree[degree == 0] = 1  # Avoid division by zero for isolated nodes

    # D^{-1/2}
    d_inv_sqrt = sp.diags(1.0 / np.sqrt(degree))

    # Normalized Laplacian: L = I - D^{-1/2} A D^{-1/2}
    identity = sp.eye(num_nodes)
    laplacian = identity - d_inv_sqrt @ adj @ d_inv_sqrt

    # Compute first m eigenvectors using eigsh
    m_actual = min(m, num_nodes - 2)
    m_actual = max(1, m_actual)  # At least 1 eigenvector

    try:
        # We want smallest eigenvalues (lowest frequencies)
        eigenvalues, eigenvectors = eigsh(laplacian, k=m_actual, which='SM', maxiter=1000)

        # Sort by eigenvalues (should already be sorted, but ensure)
        idx = np.argsort(eigenvalues)
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]

    except Exception as e:
        print(f"Warning: eigsh failed with error: {e}")
        print(f"Falling back to dense eigendecomposition for {num_nodes} nodes")
        laplacian_dense = laplacian.toarray()
        eigenvalues, eigenvectors = np.linalg.eigh(laplacian_dense)
        eigenvalues = eigenvalues[:m_actual]
        eigenvectors = eigenvectors[:, :m_actual]

    return torch.tensor(eigenvectors, dtype=torch.float32), torch.tensor(eigenvalues, dtype=torch.float32)

# ============================================================================
# SP2GNO ARCHITECTURE COMPONENTS (Section 3 of paper)
# ============================================================================

class SpectralConvLayer(nn.Module):
    """
    Spectral graph convolution layer using truncated GFT (Section 3.2.1).
    Implements: v_{j+1}^{spectral} = σ(Q_m · K ×_1 (Q_m^T · v_j(x)) + w(v_j(x)))

    Uses 3D kernel K ∈ R^{m×d×d} for enriched parameter space.
    """
    def __init__(self, in_dim, out_dim, m_eigenvectors):
        super(SpectralConvLayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.m = m_eigenvectors

        # 3D kernel for spectral filtering (m × in_dim × out_dim)
        self.spectral_kernel = nn.Parameter(torch.randn(m_eigenvectors, in_dim, out_dim) * 0.01)

        # Residual connection (w in paper)
        self.residual = nn.Linear(in_dim, out_dim)

    def forward(self, x, Q_m):
        """
        Args:
            x: Node features [N, in_dim]
            Q_m: First m eigenvectors [N, m]
        Returns:
            out: Transformed features [N, out_dim]
        """
        N, in_dim = x.shape
        m = Q_m.shape[1]

        # Ensure dimensions match
        if Q_m.shape[0] != N:
            raise ValueError(f"Q_m shape mismatch: expected [{N}, {m}], got {Q_m.shape}")

        # Graph Fourier Transform: Q_m^T · x → [m, in_dim]
        x_spectral = torch.matmul(Q_m.t(), x)

        # Mode-1 tensor-matrix product: K ×_1 x_spectral
        # Einstein notation: mpd,mp->md (contract first mode)
        x_filtered = torch.einsum('mpd,mp->md', self.spectral_kernel, x_spectral)

        # Inverse GFT: Q_m · x_filtered → [N, out_dim]
        x_out = torch.matmul(Q_m, x_filtered)

        # Add residual connection
        x_out = x_out + self.residual(x)

        # Apply activation
        out = F.gelu(x_out)

        return out


class SpatialConvLayerWithGating(nn.Module):
    """
    Spatial graph convolution with gating mechanism (Section 3.2.2).
    Implements: v_{j+1}^{spatial}_u = Σ_{v∈N(u)} γ_{uv} W v_j(x)_v

    Gating mechanism learns edge weights using Lipschitz embeddings.
    """
    def __init__(self, in_dim, out_dim, edge_dim=16, max_lipschitz_dim=64):
        super(SpatialConvLayerWithGating, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.max_lipschitz_dim = max_lipschitz_dim

        # Feature transformation W
        self.weight = nn.Linear(in_dim, out_dim)

        # W_2: Edge weight encoding
        self.edge_encoder = nn.Linear(1, edge_dim)

        # Projection layer for variable Lipschitz dimensions
        self.lipschitz_proj = nn.Linear(max_lipschitz_dim, edge_dim)

        # W_1, W_3: Gating MLP (now uses projected embeddings)
        gate_hidden = max(32, (2 * edge_dim + edge_dim) // 2)
        self.gate_mlp = nn.Sequential(
            nn.Linear(2 * edge_dim + edge_dim, gate_hidden),
            nn.ReLU(),
            nn.Linear(gate_hidden, 1),
            nn.Sigmoid()
        )

    def forward(self, x, edge_index, edge_weight, lipschitz_embed):
        """
        Args:
            x: Node features [N, in_dim]
            edge_index: Graph connectivity [2, E]
            edge_weight: Initial edge weights [E] (Euclidean distances)
            lipschitz_embed: Lipschitz embeddings [N, variable_dim]
        Returns:
            out: Aggregated features [N, out_dim]
        """
        N = x.size(0)

        # Ensure edge_index contains valid indices
        if edge_index.numel() > 0:
            max_idx = edge_index.max().item()
            if max_idx >= N:
                raise ValueError(f"Edge index contains invalid node index {max_idx} >= {N}")

        src, dst = edge_index[0], edge_index[1]

        # Ensure lipschitz embeddings match expected dimension
        if lipschitz_embed.size(0) != N:
            raise ValueError(f"Lipschitz embedding size mismatch: expected {N}, got {lipschitz_embed.size(0)}")

        # Handle empty graph case
        if edge_index.shape[1] == 0:
            return torch.zeros(N, self.out_dim, device=x.device, dtype=x.dtype)

        # Project Lipschitz embeddings to fixed dimension
        current_dim = lipschitz_embed.size(1)
        if current_dim < self.max_lipschitz_dim:
            # Pad with zeros if dimension is smaller
            padding = torch.zeros(N, self.max_lipschitz_dim - current_dim,
                                device=lipschitz_embed.device, dtype=lipschitz_embed.dtype)
            lipschitz_embed_padded = torch.cat([lipschitz_embed, padding], dim=1)
        elif current_dim > self.max_lipschitz_dim:
            # Truncate if dimension is larger
            lipschitz_embed_padded = lipschitz_embed[:, :self.max_lipschitz_dim]
        else:
            lipschitz_embed_padded = lipschitz_embed

        # Project to edge_dim
        lipschitz_proj = self.lipschitz_proj(lipschitz_embed_padded)  # [N, edge_dim]

        # Encode edge weights (W_2 * w_uv)
        edge_features = self.edge_encoder(edge_weight.unsqueeze(-1))  # [E, edge_dim]

        # Concatenate embeddings: [h_v || h_u || W_2 w_uv]
        h_src = lipschitz_proj[src]  # [E, edge_dim]
        h_dst = lipschitz_proj[dst]  # [E, edge_dim]
        gate_input = torch.cat([h_dst, h_src, edge_features], dim=-1)  # [E, 3*edge_dim]

        # Compute edge gates γ_uv (Eq. 27)
        edge_gates = self.gate_mlp(gate_input).squeeze(-1)  # [E]

        # Transform node features
        x_transformed = self.weight(x)  # [N, out_dim]

        # Message passing with gating: Γ ⊙ A v W
        messages = x_transformed[src] * edge_gates.unsqueeze(-1)  # [E, out_dim]

        # Aggregate messages
        out = torch.zeros(N, self.out_dim, device=x.device, dtype=x.dtype)
        out.index_add_(0, dst, messages)

        return out


class Sp2GNOBlock(nn.Module):
    """
    Single Sp2GNO block combining spatial and spectral convolutions (Section 3.2).
    Implements parallel processing and collaboration mechanism (Eq. 29-31).
    """
    def __init__(self, hidden_dim, m_eigenvectors, max_lipschitz_dim=64, edge_dim=16):
        super(Sp2GNOBlock, self).__init__()
        self.hidden_dim = hidden_dim

        # Spectral convolution branch
        self.spectral_conv = SpectralConvLayer(hidden_dim, hidden_dim, m_eigenvectors)

        # Spatial convolution branch with gating
        self.spatial_conv = SpatialConvLayerWithGating(hidden_dim, hidden_dim,
                                                       edge_dim=edge_dim,
                                                       max_lipschitz_dim=max_lipschitz_dim)

        # Collaboration mechanism: f([v_spectral || v_spatial])
        # Linear layer to combine both branches (Eq. 31)
        self.collaboration = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.GELU()
        )

        # Layer normalization
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, edge_index, edge_weight, Q_m, lipschitz_embed):
        """
        Args:
            x: Node features [N, hidden_dim]
            edge_index: Graph connectivity [2, E]
            edge_weight: Edge weights [E]
            Q_m: First m eigenvectors [N, m]
            lipschitz_embed: Lipschitz embeddings [N, lipschitz_dim]
        Returns:
            out: Updated node features [N, hidden_dim]
        """
        # Spectral branch (Eq. 30)
        x_spectral = self.spectral_conv(x, Q_m)

        # Spatial branch (Eq. 29)
        x_spatial = self.spatial_conv(x, edge_index, edge_weight, lipschitz_embed)

        # Concatenate outputs (Eq. 31)
        x_concat = torch.cat([x_spectral, x_spatial], dim=-1)

        # Collaboration: learn optimal combination
        x_out = self.collaboration(x_concat)

        # Residual connection and normalization
        x_out = self.norm(x_out + x)

        return x_out


class Sp2GNO(nn.Module):
    """
    Complete Spatio-Spectral Graph Neural Operator (Section 3.1).
    Architecture: P → S_1 → S_2 → ... → S_L → Q
    where S_i are Sp2GNO blocks.
    """
    def __init__(self, in_features, out_features, hidden_dim=128, num_blocks=4,
                 m_eigenvectors=32, max_lipschitz_dim=64, edge_dim=16):
        super(Sp2GNO, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.hidden_dim = hidden_dim
        self.num_blocks = num_blocks
        self.m = m_eigenvectors
        self.max_lipschitz_dim = max_lipschitz_dim

        # Uplifting layer P: R^{d_init} → R^d
        self.uplift = nn.Sequential(
            nn.Linear(in_features, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Stack of Sp2GNO blocks S_1, ..., S_L
        self.sp2gno_blocks = nn.ModuleList([
            Sp2GNOBlock(hidden_dim, m_eigenvectors, max_lipschitz_dim, edge_dim)
            for _ in range(num_blocks)
        ])

        # Downlifting layer Q: R^d → R^{d_u}
        # Two heads: one for PCA coefficients, one for uncertainty
        self.q_pca = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, out_features)
        )

        self.q_var = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.GELU(),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, data):
        """
        Args:
            data: PyG Data object with attributes:
                - x: Node features [N, in_features]
                - edge_index: Graph connectivity [2, E]
                - edge_weight: Edge weights [E]
                - Q_m: First m eigenvectors [N, m]
                - lipschitz_embed: Lipschitz embeddings [N, lipschitz_dim]
        Returns:
            pca_coeffs: Predicted PCA coefficients [N, out_features]
            variance: Predicted uncertainty [N, 1]
        """
        # Handle batched data (PyG batches graphs by concatenating them)
        # For single graphs, batch info is not present

        # Uplift features: v_0 = P({a, x})
        x = self.uplift(data.x)

        # Process through Sp2GNO blocks: v_j+1 = S_j+1(v_j)
        for block in self.sp2gno_blocks:
            x = block(x, data.edge_index, data.edge_weight,
                     data.Q_m, data.lipschitz_embed)

        # Downlift to outputs: u = Q(v_L)
        pca_coeffs = self.q_pca(x)
        variance = self.q_var(x)
        variance = F.softplus(variance) + 1e-6  # Ensure positivity

        return pca_coeffs, variance


# ============================================================================
# DATASET CLASS
# ============================================================================

class BoxGirderGraphDataset(Dataset):
    def __init__(self, root_dir, case_dirs, pca=None, train_mode=True, m_eigenvectors=M_EIGENVECTORS):
        super().__init__()
        self.root_dir = root_dir
        self.case_dirs = case_dirs
        self.pca = pca
        self.train_mode = train_mode
        self.m = m_eigenvectors
        self.data_list = []
        self._prepare()

    def _prepare(self):
        for case in self.case_dirs:
            cdir = os.path.join(self.root_dir, case)
            try:
                node_df, feats, coords, labels, df_valid = load_case(cdir)
                n_nodes = feats.shape[0]

                # Build edge index using original circular method (more suitable for box girder)
                edge_index = build_edge_index(n_nodes)

                # Compute edge weights (Euclidean distances, Eq. 23)
                edge_weight = []
                for i in range(edge_index.shape[1]):
                    src, dst = edge_index[0, i].item(), edge_index[1, i].item()
                    dist = np.linalg.norm(coords[src] - coords[dst]) + 1e-6
                    edge_weight.append(dist)
                edge_weight = torch.tensor(edge_weight, dtype=torch.float)

                # Compute first m eigenvectors of normalized Laplacian (Section 3.2.1)
                Q_m, eigenvalues = compute_normalized_laplacian_eigenvectors(
                    edge_index, n_nodes, self.m
                )

                # Compute Lipschitz embeddings (Section 3.2.2)
                lipschitz_embed = compute_lipschitz_embeddings(edge_index, n_nodes)

                # Transform labels to PCA coefficients
                if self.pca is not None:
                    phi = self.pca.transform(labels)
                else:
                    phi = None

                # Create PyG Data object
                x = torch.tensor(feats, dtype=torch.float)
                if phi is not None:
                    y = torch.tensor(phi, dtype=torch.float)
                else:
                    y = None

                data = Data(
                    x=x,
                    edge_index=edge_index,
                    edge_weight=edge_weight,
                    y=y,
                    Q_m=Q_m,
                    eigenvalues=eigenvalues,
                    lipschitz_embed=lipschitz_embed,
                    orig_labels=torch.tensor(labels, dtype=torch.float),
                    coords=torch.tensor(coords, dtype=torch.float),
                    case_name=case
                )

                # Validate dimensions
                assert x.shape[0] == Q_m.shape[0], f"Node count mismatch: x={x.shape[0]}, Q_m={Q_m.shape[0]}"
                assert x.shape[0] == lipschitz_embed.shape[0], f"Node count mismatch: x={x.shape[0]}, lipschitz={lipschitz_embed.shape[0]}"
                assert edge_index.shape[1] == edge_weight.shape[0], f"Edge count mismatch: edge_index={edge_index.shape[1]}, edge_weight={edge_weight.shape[0]}"
                if y is not None:
                    assert x.shape[0] == y.shape[0], f"Node count mismatch: x={x.shape[0]}, y={y.shape[0]}"

                self.data_list.append(data)

            except Exception as e:
                print(f"Warning: Could not load case {case}: {e}")
                import traceback
                traceback.print_exc()
                continue

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

# ============================================================================
# LOSS FUNCTIONS
# ============================================================================

class PCALoss(nn.Module):
    """Loss for PCA coefficient prediction"""
    def __init__(self, m_components, use_weights=True):
        super().__init__()
        self.m_components = m_components
        self.use_weights = use_weights

    def forward(self, pred_phi, target_phi, variance=None, pca_weights=None):
        # MSE loss for PCA coefficients
        squared_errors = (pred_phi - target_phi) ** 2

        # Weight by PCA explained variance
        if self.use_weights and pca_weights is not None:
            weights = pca_weights.to(pred_phi.device)
            weighted_errors = squared_errors * weights
        else:
            weighted_errors = squared_errors

        loss_pca = weighted_errors.mean()

        # Variance loss (negative log-likelihood)
        if variance is not None:
            nll = 0.5 * (squared_errors / variance + torch.log(variance))
            loss_var = nll.mean()
            return loss_pca + 0.1 * loss_var

        return loss_pca

class SpatialSmoothnessLoss(nn.Module):
    """Regularization for spatial smoothness"""
    def __init__(self):
        super().__init__()

    def forward(self, pca_pred, edge_index):
        src, dst = edge_index[0], edge_index[1]
        diff = pca_pred[src] - pca_pred[dst]
        smoothness = (diff ** 2).mean()
        return smoothness

# ============================================================================
# TRAINING AND EVALUATION
# ============================================================================

def train_epoch(model, loader, optimizer, loss_fn, smooth_loss_fn, device, pca_weights=None):
    model.train()
    total_loss = 0
    num_batches = 0

    for data in loader:
        try:
            data = data.to(device)
            optimizer.zero_grad()

            pca_pred, var_pred = model(data)
            target = data.y

            # Main loss
            loss = loss_fn(pca_pred, target, var_pred, pca_weights)

            # Spatial smoothness regularization
            smooth_loss = smooth_loss_fn(pca_pred, data.edge_index)

            # Total loss
            total_loss_batch = loss + 0.01 * smooth_loss
            total_loss_batch.backward()

            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += total_loss_batch.item()
            num_batches += 1

        except Exception as e:
            print(f"Error in training batch: {e}")
            import traceback
            traceback.print_exc()
            continue

    return total_loss / max(num_batches, 1)

def evaluate(model, dataset, pca, device):
    model.eval()
    results = {}

    with torch.no_grad():
        for data in dataset.data_list:
            try:
                data = data.to(device)
                pca_pred, var_pred = model(data)

                pca_pred = pca_pred.cpu().numpy()
                var_pred = var_pred.cpu().numpy()

                # Reconstruct time series
                recon = pca.inverse_transform(pca_pred)
                true_ts = data.orig_labels.cpu().numpy()

                n_nodes = true_ts.shape[0]
                rmse_list = []
                r2_list = []

                for i in range(n_nodes):
                    y_true = true_ts[i]
                    y_pred = recon[i]
                    rmse_list.append(math.sqrt(mean_squared_error(y_true, y_pred)))
                    r2_list.append(r2_score(y_true, y_pred))

                # Peak metrics
                true_peaks = true_ts.max(axis=1)
                pred_peaks = recon.max(axis=1)
                peak_mape = np.mean(np.abs((true_peaks - pred_peaks) / (true_peaks + 1e-8)))

                # Impulse metrics
                true_impulses = np.array([np.trapz(ts) for ts in true_ts])
                pred_impulses = np.array([np.trapz(ts) for ts in recon])
                impulse_mape = np.mean(np.abs((true_impulses - pred_impulses) / (true_impulses + 1e-8)))

                overall_r2 = np.mean(r2_list)

                results[data.case_name] = {
                    'rmse_mean': float(np.mean(rmse_list)),
                    'r2_mean': float(overall_r2),
                    'peak_mape': float(peak_mape),
                    'impulse_mape': float(impulse_mape),
                    'variance_mean': float(np.mean(var_pred))
                }

            except Exception as e:
                print(f"Error evaluating case {data.case_name}: {e}")
                import traceback
                traceback.print_exc()
                continue

    return results

# ============================================================================
# VISUALIZATION
# ============================================================================

def visualize_results(model, test_dataset, pca, device, results):
    """Generate comprehensive visualizations"""

    model.eval()

    if len(test_dataset) == 0:
        print("No test cases to visualize")
        return

    sample_data = test_dataset.get(0)
    case_name = sample_data.case_name

    print(f"Visualizing case: {case_name}")

    try:
        with torch.no_grad():
            sample_data = sample_data.to(device)
            pca_pred, var_pred = model(sample_data)

            pca_pred = pca_pred.cpu().numpy()
            var_pred = var_pred.cpu().numpy()

            # Reconstruct time series
            recon = pca.inverse_transform(pca_pred)
            true_ts = sample_data.orig_labels.cpu().numpy()
            coords = sample_data.coords.cpu().numpy()

    except Exception as e:
        print(f"Error during visualization: {e}")
        import traceback
        traceback.print_exc()
        return

    # Create figure with subplots
    fig = plt.figure(figsize=(20, 12))

    # 1. Time series comparison
    ax1 = plt.subplot(2, 3, 1)
    r2_scores = [r2_score(true_ts[i], recon[i]) for i in range(len(true_ts))]
    best_idx = np.argmax(r2_scores)
    worst_idx = np.argmin(r2_scores)

    time_points = np.arange(len(true_ts[0]))
    ax1.plot(time_points, true_ts[best_idx], 'b-', label=f'True (Best R²={r2_scores[best_idx]:.3f})', linewidth=2)
    ax1.plot(time_points, recon[best_idx], 'r--', label='Predicted (Best)', linewidth=2)
    ax1.plot(time_points, true_ts[worst_idx], 'g-', label=f'True (Worst R²={r2_scores[worst_idx]:.3f})', linewidth=1, alpha=0.7)
    ax1.plot(time_points, recon[worst_idx], 'm--', label='Predicted (Worst)', linewidth=1, alpha=0.7)
    ax1.set_xlabel('Time Step')
    ax1.set_ylabel('Overpressure')
    ax1.set_title(f'Time Series Comparison\n{case_name}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Spatial distribution of R² scores
    ax2 = plt.subplot(2, 3, 2)
    scatter = ax2.scatter(coords[:, 0], coords[:, 1], c=r2_scores,
                         cmap='RdYlGn', s=100, vmin=0, vmax=1, edgecolors='black')
    ax2.set_xlabel('X Coordinate')
    ax2.set_ylabel('Y Coordinate')
    ax2.set_title('Spatial Distribution of R² Scores')
    plt.colorbar(scatter, ax=ax2, label='R² Score')

    # 3. Spatial distribution of predicted variance
    ax3 = plt.subplot(2, 3, 3)
    scatter = ax3.scatter(coords[:, 0], coords[:, 1], c=var_pred.flatten(),
                         cmap='viridis', s=100, edgecolors='black')
    ax3.set_xlabel('X Coordinate')
    ax3.set_ylabel('Y Coordinate')
    ax3.set_title('Predicted Uncertainty (Variance)')
    plt.colorbar(scatter, ax=ax3, label='Variance')

    # 4. PCA coefficients comparison
    ax4 = plt.subplot(2, 3, 4)
    avg_true_pca = pca.transform(true_ts).mean(axis=0)
    avg_pred_pca = pca_pred.mean(axis=0)
    x_pos = np.arange(len(avg_true_pca))
    width = 0.35
    ax4.bar(x_pos - width/2, avg_true_pca, width, label='True', alpha=0.7)
    ax4.bar(x_pos + width/2, avg_pred_pca, width, label='Predicted', alpha=0.7)
    ax4.set_xlabel('PCA Component')
    ax4.set_ylabel('Average Coefficient Value')
    ax4.set_title('PCA Coefficients Comparison')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # 5. Peak overpressure comparison
    ax5 = plt.subplot(2, 3, 5)
    true_peaks = true_ts.max(axis=1)
    pred_peaks = recon.max(axis=1)
    ax5.scatter(true_peaks, pred_peaks, alpha=0.6, s=50)
    max_val = max(max(true_peaks), max(pred_peaks))
    ax5.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    ax5.set_xlabel('True Peak Overpressure')
    ax5.set_ylabel('Predicted Peak Overpressure')
    ax5.set_title('Peak Overpressure: True vs Predicted')
    ax5.legend()
    ax5.grid(True, alpha=0.3)

    # 6. R² distribution
    ax6 = plt.subplot(2, 3, 6)
    ax6.hist(r2_scores, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    ax6.axvline(np.mean(r2_scores), color='red', linestyle='--', linewidth=2,
               label=f'Mean: {np.mean(r2_scores):.3f}')
    ax6.set_xlabel('R² Score')
    ax6.set_ylabel('Frequency')
    ax6.set_title('Distribution of R² Scores Across Nodes')
    ax6.legend()
    ax6.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('sp2gno_results.png', dpi=300, bbox_inches='tight')
    print("Visualization saved to: sp2gno_results.png")
    plt.close()

    # Additional plot: Overall performance across all test cases
    if len(results) > 1:
        fig2, axes = plt.subplots(2, 2, figsize=(15, 10))

        case_names = list(results.keys())
        r2_values = [results[c]['r2_mean'] for c in case_names]
        rmse_values = [results[c]['rmse_mean'] for c in case_names]
        peak_mape_values = [results[c]['peak_mape'] for c in case_names]
        impulse_mape_values = [results[c]['impulse_mape'] for c in case_names]

        axes[0, 0].bar(range(len(case_names)), r2_values, color='skyblue', alpha=0.7)
        axes[0, 0].set_xlabel('Test Case')
        axes[0, 0].set_ylabel('R² Score')
        axes[0, 0].set_title('R² Score by Test Case')
        axes[0, 0].set_xticks(range(len(case_names)))
        axes[0, 0].set_xticklabels(case_names, rotation=45, ha='right')
        axes[0, 0].grid(True, alpha=0.3)

        axes[0, 1].bar(range(len(case_names)), rmse_values, color='lightcoral', alpha=0.7)
        axes[0, 1].set_xlabel('Test Case')
        axes[0, 1].set_ylabel('RMSE')
        axes[0, 1].set_title('RMSE by Test Case')
        axes[0, 1].set_xticks(range(len(case_names)))
        axes[0, 1].set_xticklabels(case_names, rotation=45, ha='right')
        axes[0, 1].grid(True, alpha=0.3)

        axes[1, 0].bar(range(len(case_names)), peak_mape_values, color='lightgreen', alpha=0.7)
        axes[1, 0].set_xlabel('Test Case')
        axes[1, 0].set_ylabel('Peak MAPE')
        axes[1, 0].set_title('Peak MAPE by Test Case')
        axes[1, 0].set_xticks(range(len(case_names)))
        axes[1, 0].set_xticklabels(case_names, rotation=45, ha='right')
        axes[1, 0].grid(True, alpha=0.3)

        axes[1, 1].bar(range(len(case_names)), impulse_mape_values, color='gold', alpha=0.7)
        axes[1, 1].set_xlabel('Test Case')
        axes[1, 1].set_ylabel('Impulse MAPE')
        axes[1, 1].set_title('Impulse MAPE by Test Case')
        axes[1, 1].set_xticks(range(len(case_names)))
        axes[1, 1].set_xticklabels(case_names, rotation=45, ha='right')
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('sp2gno_overall_performance.png', dpi=300, bbox_inches='tight')
        print("Overall performance plot saved to: sp2gno_overall_performance.png")
        plt.close()

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    print("="*70)
    print("Sp2GNO for PCA-compressed Time Series Prediction")
    print("Exact Architecture from Paper - FIXED VERSION")
    print("="*70)
    print(f"PyTorch version: {torch.__version__}")
    print(f"Device: {DEVICE}")
    print(f"Random seed: {SEED}")
    print("="*70)

    # Extract data
    if not os.path.exists(ZIP_FILE_PATH):
        print(f"Error: Zip file not found at {ZIP_FILE_PATH}")
        print("Please update ZIP_FILE_PATH variable with the correct path.")
        return

    data_root = extract_zip(ZIP_FILE_PATH)
    case_dirs = find_case_directories(data_root)
    print(f"\nFound {len(case_dirs)} cases")

    if not case_dirs:
        print("No valid case directories found!")
        return

    # Split train/test
    if len(case_dirs) >= 12:
        random.shuffle(case_dirs)
        train_cases = case_dirs[:8]
        test_cases = case_dirs[8:]
    else:
        n_train = max(1, int(0.66 * len(case_dirs)))
        random.shuffle(case_dirs)
        train_cases = case_dirs[:n_train]
        test_cases = case_dirs[n_train:]

    print(f"Training cases: {len(train_cases)}")
    print(f"Test cases: {len(test_cases)}")

    # Fit PCA on training data
    print("\n" + "="*70)
    print("STEP 1: Fitting PCA on training data")
    print("="*70)

    all_train_labels = []
    successful_train_cases = []

    for case in train_cases:
        try:
            _, _, _, labels, _ = load_case(os.path.join(data_root, case))
            all_train_labels.append(labels)
            successful_train_cases.append(case)
        except Exception as e:
            print(f"Warning: Could not load training case {case}: {e}")
            continue

    if not all_train_labels:
        print("No training data could be loaded!")
        return

    train_cases = successful_train_cases
    all_train_labels = np.vstack(all_train_labels)
    print(f"Training label matrix shape: {all_train_labels.shape}")

    # Fit PCA
    pca = PaperPCA(n_components=PCA_COMPONENTS)
    pca.fit(all_train_labels)

    total_variance = np.var(all_train_labels, axis=0).sum()
    explained_variance_ratio = pca.explained_variance_ / total_variance
    print(f"Explained variance (first 5): {explained_variance_ratio[:5].round(4)}")
    print(f"Cumulative explained variance: {np.sum(explained_variance_ratio):.4f}")

    # Create PCA weights for loss function
    pca_weights = torch.tensor(explained_variance_ratio, dtype=torch.float32)

    # Create datasets
    print("\n" + "="*70)
    print("STEP 2: Creating graph datasets with Sp2GNO preprocessing")
    print("="*70)

    train_dataset = BoxGirderGraphDataset(
        data_root, train_cases, pca=pca, train_mode=True, m_eigenvectors=M_EIGENVECTORS
    )
    test_dataset = BoxGirderGraphDataset(
        data_root, test_cases, pca=pca, train_mode=False, m_eigenvectors=M_EIGENVECTORS
    )

    if len(train_dataset) == 0:
        print("No training data available!")
        return

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")

    # Create data loaders - Use batch_size=1 because each graph has unique structure
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Get sample to determine dimensions
    sample = train_dataset.get(0)
    in_features = sample.x.shape[1]
    num_nodes = sample.x.shape[0]

    # Determine max Lipschitz dimension across all graphs
    max_lipschitz_dim = 0
    for data in train_dataset.data_list:
        max_lipschitz_dim = max(max_lipschitz_dim, data.lipschitz_embed.shape[1])
    for data in test_dataset.data_list:
        max_lipschitz_dim = max(max_lipschitz_dim, data.lipschitz_embed.shape[1])

    edge_dim = 16  # Fixed edge feature dimension

    print(f"\nInput features: {in_features}")
    print(f"Number of nodes (sample): {num_nodes}")
    print(f"Output PCA components: {PCA_COMPONENTS}")
    print(f"Max Lipschitz embedding dimension: {max_lipschitz_dim}")
    print(f"Edge feature dimension: {edge_dim}")
    print(f"Number of eigenvectors (m): {M_EIGENVECTORS}")

    # Initialize model
    print("\n" + "="*70)
    print("STEP 3: Initializing Sp2GNO model (Paper Architecture)")
    print("="*70)

    model = Sp2GNO(
        in_features=in_features,
        out_features=PCA_COMPONENTS,
        hidden_dim=WIDTH,
        num_blocks=NUM_SP2GNO_BLOCKS,
        m_eigenvectors=M_EIGENVECTORS,
        max_lipschitz_dim=max_lipschitz_dim,
        edge_dim=edge_dim
    ).to(DEVICE)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Number of Sp2GNO blocks: {NUM_SP2GNO_BLOCKS}")
    print(f"Hidden dimension: {WIDTH}")
    print(f"Device: {DEVICE}")

    # Initialize optimizer and loss functions
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=20
    )

    loss_fn = PCALoss(m_components=PCA_COMPONENTS, use_weights=True)
    smooth_loss_fn = SpatialSmoothnessLoss()

    # Training loop
    print("\n" + "="*70)
    print("STEP 4: Training Sp2GNO")
    print("="*70)

    best_loss = float('inf')
    patience = 50
    patience_counter = 0

    for epoch in range(1, EPOCHS + 1):
        train_loss = train_epoch(
            model, train_loader, optimizer, loss_fn,
            smooth_loss_fn, DEVICE, pca_weights
        )

        scheduler.step(train_loss)

        if train_loss < best_loss:
            best_loss = train_loss
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss,
            }, "best_sp2gno_model.pt")
            if epoch % 10 == 0:
                print(f"✓ New best model saved at epoch {epoch} with loss {best_loss:.6f}")
        else:
            patience_counter += 1

        if epoch % 50 == 0 or epoch == 1:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch}/{EPOCHS} - Loss: {train_loss:.6f} - Best: {best_loss:.6f} - LR: {current_lr:.2e}")

        # Early stopping
        if patience_counter >= patience and epoch > 200:
            print(f"\nEarly stopping at epoch {epoch}")
            break

    # Load best model
    checkpoint = torch.load("best_sp2gno_model.pt")
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"\nLoaded best model from epoch {checkpoint['epoch']}")

    # Evaluation
    print("\n" + "="*70)
    print("STEP 5: Evaluation on test set")
    print("="*70)

    results = evaluate(model, test_dataset, pca, DEVICE)

    print("\nTest Results:")
    print("-" * 70)
    for case_name, metrics in results.items():
        print(f"{case_name}:")
        print(f"  R²: {metrics['r2_mean']:.4f}")
        print(f"  RMSE: {metrics['rmse_mean']:.4f}")
        print(f"  Peak MAPE: {metrics['peak_mape']:.4f}")
        print(f"  Impulse MAPE: {metrics['impulse_mape']:.4f}")
        print(f"  Avg Variance: {metrics['variance_mean']:.4f}")
        print()

    # Overall statistics
    if results:
        avg_r2 = np.mean([m['r2_mean'] for m in results.values()])
        avg_rmse = np.mean([m['rmse_mean'] for m in results.values()])
        avg_peak_mape = np.mean([m['peak_mape'] for m in results.values()])
        avg_impulse_mape = np.mean([m['impulse_mape'] for m in results.values()])

        print("="*70)
        print("OVERALL TEST PERFORMANCE")
        print("="*70)
        print(f"Average R²: {avg_r2:.4f}")
        print(f"Average RMSE: {avg_rmse:.4f}")
        print(f"Average Peak MAPE: {avg_peak_mape:.4f}")
        print(f"Average Impulse MAPE: {avg_impulse_mape:.4f}")

    # Save model and PCA
    torch.save(model.state_dict(), "sp2gno_model.pt")
    joblib.dump(pca, "sp2gno_pca.pkl")
    print("\nModel saved to: sp2gno_model.pt")
    print("PCA saved to: sp2gno_pca.pkl")

    # Visualization
    print("\n" + "="*70)
    print("STEP 6: Generating visualizations")
    print("="*70)

    visualize_results(model, test_dataset, pca, DEVICE, results)

    print("\n" + "="*70)
    print("TRAINING COMPLETE!")
    print("="*70)

if __name__ == "__main__":
    main()

Sp2GNO for PCA-compressed Time Series Prediction
Exact Architecture from Paper - FIXED VERSION
PyTorch version: 2.8.0+cu126
Device: cpu
Random seed: 42
Extracting /content/box.zip to /tmp/tmp1wplm40n...
Extraction complete. Data root: /tmp/tmp1wplm40n/Dataset for box girder

Found 12 cases
Training cases: 8
Test cases: 4

STEP 1: Fitting PCA on training data
Training label matrix shape: (624, 8231)
Explained variance (first 5): [0.8087 0.0963 0.0287 0.0188 0.0164]
Cumulative explained variance: 1.0000

STEP 2: Creating graph datasets with Sp2GNO preprocessing
Train dataset size: 8
Test dataset size: 4

Input features: 5
Number of nodes (sample): 70
Output PCA components: 20
Max Lipschitz embedding dimension: 20
Edge feature dimension: 16
Number of eigenvectors (m): 16

STEP 3: Initializing Sp2GNO model (Paper Architecture)
Model parameters: 264,744
Number of Sp2GNO blocks: 3
Hidden dimension: 64
Device: cpu

STEP 4: Training Sp2GNO
Epoch 1/3000 - Loss: 5.364321 - Best: 5.364321 - LR: 5

/tmp/ipython-input-2920663278.py:790: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  true_impulses = np.array([np.trapz(ts) for ts in true_ts])
/tmp/ipython-input-2920663278.py:791: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  pred_impulses = np.array([np.trapz(ts) for ts in recon])
/tmp/ipython-input-2920663278.py:790: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  true_impulses = np.array([np.trapz(ts) for ts in true_ts])
/tmp/ipython-input-2920663278.py:791: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  pred_impulses = np.array([np.trapz(ts) for ts in recon])
/tmp/ipython-input-2920663278.py:790: DeprecationWarning: `trapz


Test Results:
----------------------------------------------------------------------
26.6+1.9:
  R²: 0.7799
  RMSE: 0.0318
  Peak MAPE: 0.1267
  Impulse MAPE: 0.0892
  Avg Variance: 0.1006

20.6+1.9:
  R²: 0.8224
  RMSE: 0.0336
  Peak MAPE: 0.1547
  Impulse MAPE: 0.1258
  Avg Variance: 0.1786

20.6+10:
  R²: 0.7903
  RMSE: 0.0272
  Peak MAPE: 0.1357
  Impulse MAPE: 0.7684
  Avg Variance: 0.0657

31.3+15:
  R²: 0.7323
  RMSE: 0.0226
  Peak MAPE: 0.1944
  Impulse MAPE: 6.1567
  Avg Variance: 0.0448

OVERALL TEST PERFORMANCE
Average R²: 0.7812
Average RMSE: 0.0288
Average Peak MAPE: 0.1529
Average Impulse MAPE: 1.7850

Model saved to: sp2gno_model.pt
PCA saved to: sp2gno_pca.pkl

STEP 6: Generating visualizations
Visualizing case: 26.6+1.9
Visualization saved to: sp2gno_results.png
Overall performance plot saved to: sp2gno_overall_performance.png

TRAINING COMPLETE!
